# 0. Imports

In [1]:
import os
import warnings

import pandas as pd
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# 1. Fine-Dust Data in Seoul

The original data was downloaded from [Public Data Portal](https://www.data.go.kr/data/15089266/fileData.do). The data contains hourly data for fine dust measured in Seoul, Korea, for the period of 2008 to 2021.

In [2]:
os.listdir("../data/raw/fine_dust")

['서울시 대기질 자료 제공_2008-2011.csv',
 '서울시 대기질 자료 제공_2012-2015.csv',
 '서울시 대기질 자료 제공_2016-2019.csv',
 '서울시 대기질 자료 제공_2020-2021.csv']

In [3]:
fine_dust_df = pd.DataFrame()
for i in range(len(os.listdir("../data/raw/fine_dust"))):
    df = pd.read_csv(
        f"../data/raw/fine_dust/{os.listdir('../data/raw/fine_dust')[i]}",
        encoding="cp949",
    )
    fine_dust_df = pd.concat([fine_dust_df, df])

In [4]:
# Column names were renamed in English

fine_dust_df = fine_dust_df.rename(
    columns={
        "일시": "date",
        "구분": "data_Type",
        "미세먼지(PM10)": "PM10_Counts",
        "초미세먼지(PM25)": "PM25_Counts",
    }
)

In [5]:
fine_dust_df.head(3)

,date,data_Type,PM10_Counts,PM25_Counts
0,2011-12-31 23:00,평균,89.0,61.0
1,2011-12-31 23:00,강남구,91.0,58.0
2,2011-12-31 23:00,강동구,89.0,59.0


`PM10_Counts` and `PM25_Counts` are densities of particulate matter less than 10um and 2.5um in microgram/cubic meter, respectively. The density of PM2.5 was written as `PM25_Counts` on purpose since the dot in the middle of feature name can cause unwanted results.

In [6]:
# The data averaged over Seoul were only selected.
fine_dust_df = (
    fine_dust_df.query("data_Type == '평균'")  # '평균' means Average
    .sort_values(by="date")
    .drop(columns=["data_Type"])
    .reset_index(drop=True)
)

In [7]:
fine_dust_df["date"] = pd.to_datetime(fine_dust_df["date"])

In [8]:
fine_dust_df.head(3)

,date,PM10_Counts,PM25_Counts
0,2008-01-01 10:00:00,30.0,11.0
1,2008-01-01 11:00:00,29.0,13.0
2,2008-01-01 12:00:00,29.0,12.0


In [9]:
fine_dust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122733 entries, 0 to 122732
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         122733 non-null  datetime64[ns]
 1   PM10_Counts  122733 non-null  float64       
 2   PM25_Counts  122733 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 2.8 MB


# 2. Seoul Weather Data

Weather data was downloaded from [Open MET Data Portal](https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36&tabNo=1) by KMA (Korea Meteorological Administration) Weather Data Service.

In [10]:
os.listdir("../data/raw/weather")

['SURFACE_ASOS_108_HR_2008_2008_2015.csv',
 'SURFACE_ASOS_108_HR_2009_2009_2015.csv',
 'SURFACE_ASOS_108_HR_2010_2010_2018.csv',
 'SURFACE_ASOS_108_HR_2011_2011_2018.csv',
 'SURFACE_ASOS_108_HR_2012_2012_2018.csv',
 'SURFACE_ASOS_108_HR_2013_2013_2018.csv',
 'SURFACE_ASOS_108_HR_2014_2014_2018.csv',
 'SURFACE_ASOS_108_HR_2015_2015_2018.csv',
 'SURFACE_ASOS_108_HR_2016_2016_2017.csv',
 'SURFACE_ASOS_108_HR_2017_2017_2018.csv',
 'SURFACE_ASOS_108_HR_2018_2018_2019.csv',
 'SURFACE_ASOS_108_HR_2019_2019_2020.csv',
 'SURFACE_ASOS_108_HR_2020_2020_2021.csv',
 'SURFACE_ASOS_108_HR_2021_2021_2022.csv']

In [11]:
weather_df = pd.DataFrame()
for i in range(len(os.listdir("../data/raw/weather"))):
    df = pd.read_csv(
        f"../data/raw/weather/{os.listdir('../data/raw/weather')[i]}", encoding="cp949"
    )
    weather_df = pd.concat([weather_df, df])

In [12]:
weather_df.columns

Index(['지점', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)',
       '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)', '중하층운량(10분위)',
       '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)', '현상번호(국내식)',
       '지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)',
       '30cm 지중온도(°C)'],
      dtype='object')

In [13]:
# Columns were filtered according to the relevance.

relevant_cols = [
    "일시",
    "기온(°C)",
    "강수량(mm)",
    "풍속(m/s)",
    "풍향(16방위)",
    "습도(%)",
    "증기압(hPa)",
    "이슬점온도(°C)",
    "현지기압(hPa)",
    "전운량(10분위)",
    "최저운고(100m )",
]

weather_df = weather_df[relevant_cols]

In [14]:
# Column names were renamed in English

cols_in_english = [
    "date",
    "temp(°C)",
    "precipitation(mm)",
    "wind_speed(m/s)",
    "wind_direction",
    "humidity(%)",
    "vapor_P(hPa)",
    "dew_point_temp(°C)",
    "local_P(hPa)",
    "cloud_cover",
    "lowest_ceiling(100m)",
]

for i, col in enumerate(relevant_cols):
    weather_df.rename(columns={relevant_cols[i]: cols_in_english[i]}, inplace=True)

In [15]:
weather_df["date"] = pd.to_datetime(weather_df["date"])

In [16]:
weather_df = weather_df.sort_values(by="date").reset_index(drop=True)

In [17]:
weather_df.head(3)

,date,temp(°C),precipitation(mm),wind_speed(m/s),wind_direction,humidity(%),vapor_P(hPa),dew_point_temp(°C),local_P(hPa),cloud_cover,lowest_ceiling(100m)
0,2008-01-01 00:00:00,-7.0,NaN,3.5,270.0,52.0,1.8,-16.1,1008.0,1.0,10.0
1,2008-01-01 01:00:00,-7.3,NaN,4.9,290.0,52.0,NaN,NaN,NaN,NaN,NaN
2,2008-01-01 02:00:00,-7.5,NaN,4.1,270.0,52.0,NaN,NaN,NaN,NaN,NaN


In [18]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122750 entries, 0 to 122749
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   date                  122750 non-null  datetime64[ns]
 1   temp(°C)              122728 non-null  float64       
 2   precipitation(mm)     12267 non-null   float64       
 3   wind_speed(m/s)       122677 non-null  float64       
 4   wind_direction        122677 non-null  float64       
 5   humidity(%)           122711 non-null  float64       
 6   vapor_P(hPa)          120884 non-null  float64       
 7   dew_point_temp(°C)    120879 non-null  float64       
 8   local_P(hPa)          120868 non-null  float64       
 9   cloud_cover           102288 non-null  float64       
 10  lowest_ceiling(100m)  59433 non-null   float64       
dtypes: datetime64[ns](1), float64(10)
memory usage: 10.3 MB


# 3. Data Merge

In [19]:
combined_df = fine_dust_df.merge(weather_df, on="date", how="left")

In [20]:
combined_df.head(3)

,date,PM10_Counts,PM25_Counts,temp(°C),precipitation(mm),wind_speed(m/s),wind_direction,humidity(%),vapor_P(hPa),dew_point_temp(°C),local_P(hPa),cloud_cover,lowest_ceiling(100m)
0,2008-01-01 10:00:00,30.0,11.0,-7.6,NaN,4.6,340.0,47.0,NaN,NaN,NaN,NaN,NaN
1,2008-01-01 11:00:00,29.0,13.0,-6.9,NaN,4.1,290.0,42.0,NaN,NaN,NaN,NaN,NaN
2,2008-01-01 12:00:00,29.0,12.0,-5.8,NaN,4.1,290.0,42.0,1.6,-17.5,1010.7,2.0,12.0


In [21]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122733 entries, 0 to 122732
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   date                  122733 non-null  datetime64[ns]
 1   PM10_Counts           122733 non-null  float64       
 2   PM25_Counts           122733 non-null  float64       
 3   temp(°C)              122725 non-null  float64       
 4   precipitation(mm)     12267 non-null   float64       
 5   wind_speed(m/s)       122674 non-null  float64       
 6   wind_direction        122674 non-null  float64       
 7   humidity(%)           122708 non-null  float64       
 8   vapor_P(hPa)          120881 non-null  float64       
 9   dew_point_temp(°C)    120876 non-null  float64       
 10  local_P(hPa)          120865 non-null  float64       
 11  cloud_cover           102285 non-null  float64       
 12  lowest_ceiling(100m)  59431 non-null   float64       
dtyp

In [22]:
combined_df.to_csv("../data/interim/seoul_fine_dust_weather_2008_2021.csv", index=False)